# Predicting News Category With BERT IN Tensorflow

---

Bidirectional Encoder Representations from Transformers or BERT for short is a very popular NLP model from Google known for producing state-of-the-art results in a wide variety of NLP tasks.

The importance of Natural Language Processing(NLP) is profound in the Artificial Intelligence domain. The most abundant data in the world today is in the form of texts and having a powerful text processing system is critical and is more than  just a necessity.

In this article we look at implementing a multi-class classification using the state-of-the-art model, BERT.

---

##### Pre-Requisites:

##### An Understanding of BERT
---

## About Dataset

For this article, we will use MachineHack’s Predict The News Category Hackathon data. The data  consists of a collection of news articles which are categorized into four sections. The features of the datasets are as follows:

Size of training set: 7,628 records
Size of test set: 2,748 records

FEATURES:

STORY:  A part of the main content of the article to be published as a piece of news.
SECTION: The genre/category the STORY falls in.

There are four distinct sections where each story may fall in to. The Sections are labelled as follows :
Politics: 0
Technology: 1
Entertainment: 2
Business: 3


## Importing Necessary Libraries

In [123]:
from datetime import datetime

import pandas as pd

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tensorflow_hub as hub
import bert

from sklearn.model_selection import train_test_split

## Setting The Output Directory
---
While fine-tuning the model, we will save the training checkpoints and the model in an output directory so that we can use the trained model for our predictions later.

The following code block sets an output directory :

In [125]:
OUTPUT_DIR = 'bert'

## Loading The Data
---
We will now load the data from a Google Drive directory and will also split the training set in to training and validation sets.


In [126]:
DATA_SIZE = 1000
data = pd.read_csv("../nlp-for-future-data/reviews" + str(DATA_SIZE) + ".csv")
train, test = train_test_split(data, test_size=0.2, random_state=100)

In [127]:
print("Training Set Shape :", train.shape)
print("Test Set Shape :", test.shape)

Training Set Shape : (800, 3)
Test Set Shape : (200, 3)


In [128]:
DATA_COLUMN = 'review'
LABEL_COLUMN = 'score'
label_list = list(train.score.unique())

## Data Preprocessing

BERT model accept only a specific type of input and the datasets are usually structured to have have the following four features:

* guid : A unique id that represents an observation.
* text_a : The text we need to classify into given categories
* text_b: It is used when we're training a model to understand the relationship between sentences and it does not apply for classification problems.
* label: It consists of the labels or classes or categories that a given text belongs to.
 
In our dataset we have text_a and label. The following code block will create objects for each of the above mentioned features for all the records in our dataset using the InputExample class provided in the BERT library.


In [129]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                             text_a=x[DATA_COLUMN],
                                                                             text_b=None,
                                                                             label=x[LABEL_COLUMN]), axis=1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                          text_a=x[DATA_COLUMN],
                                                                          text_b=None,
                                                                          label=x[LABEL_COLUMN]), axis=1)

In [130]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  I found this book to be exceptionally frustrating to read. My frustration did not stem from the author's ability to create a solid story. In fact, it is very evident from the text that Caldwell is an incredibly gifted writer. His depiction of the protagonist being tortured put chills down my spine. The twists of logic that were expressed by the torturer in those scenes were exactly what one would imagine someone in that position saying to justify their behavior. Caldwell is also able to show in his writing that he has a strong understanding of the philosophical and theological arguments used to support a Christian lifestyle.Given his ability to write meaningful narrative and realistic dialogue, it is tremendously disappointing that Caldwell chose to waste his gifts by creating such an unlikable protagonist. I recognize that having a central character with a &quot;fatal flaw&quot; is a standard literary d

We will now get down to business with the pretrained BERT.  In this example we will use the ```bert_uncased_L-12_H-768_A-12/1``` model. To check all available versions click [here](https://tfhub.dev/s?network-architecture=transformer&publisher=google).

We will be using the vocab.txt file in the model to map the words in the dataset to indexes. Also the loaded BERT model is trained on uncased/lowercase data and hence the data we feed to train the model should also be of lowercase.

---

The following code block loads the pre-trained BERT model and initializers a tokenizer object for tokenizing the texts.


In [131]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


tokenizer = create_tokenizer_from_hub_module()

In [132]:
#Here is what the tokenised sample of the first training set observation looks like.
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['i', 'found', 'this', 'book', 'to', 'be', 'exceptionally', 'frustrating', 'to', 'read', '.', 'my', 'frustration', 'did', 'not', 'stem', 'from', 'the', 'author', "'", 's', 'ability', 'to', 'create', 'a', 'solid', 'story', '.', 'in', 'fact', ',', 'it', 'is', 'very', 'evident', 'from', 'the', 'text', 'that', 'caldwell', 'is', 'an', 'incredibly', 'gifted', 'writer', '.', 'his', 'depiction', 'of', 'the', 'protagonist', 'being', 'tortured', 'put', 'chill', '##s', 'down', 'my', 'spine', '.', 'the', 'twists', 'of', 'logic', 'that', 'were', 'expressed', 'by', 'the', 'torture', '##r', 'in', 'those', 'scenes', 'were', 'exactly', 'what', 'one', 'would', 'imagine', 'someone', 'in', 'that', 'position', 'saying', 'to', 'justify', 'their', 'behavior', '.', 'caldwell', 'is', 'also', 'able', 'to', 'show', 'in', 'his', 'writing', 'that', 'he', 'has', 'a', 'strong', 'understanding', 'of', 'the', 'philosophical', 'and', 'theological', 'arguments', 'used', 'to', 'support', 'a', 'christian', 'lifestyle', '.

We will now format out text in to input features which the BERT model expects. We will also set a sequence length which will be the length of the input features.

In [133]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples,
                                                                  label_list,
                                                                  MAX_SEQ_LENGTH,
                                                                  tokenizer)

test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples,
                                                                 label_list,
                                                                 MAX_SEQ_LENGTH,
                                                                 tokenizer)

In [134]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-" * 30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-" * 30)
print("Input IDs : ", train_features[0].input_ids)
print("-" * 30)
print("Input Masks : ", train_features[0].input_mask)
print("-" * 30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  I found this book to be exceptionally frustrating to read. My frustration did not stem from the author's ability to create a solid story. In fact, it is very evident from the text that Caldwell is an incredibly gifted writer. His depiction of the protagonist being tortured put chills down my spine. The twists of logic that were expressed by the torturer in those scenes were exactly what one would imagine someone in that position saying to justify their behavior. Caldwell is also able to show in his writing that he has a strong understanding of the philosophical and theological arguments used to support a Christian lifestyle.Given his ability to write meaningful narrative and realistic dialogue, it is tremendously disappointing that Caldwell chose to waste his gifts by creating such an unlikable protagonist. I recognize that having a central character with a &quot;fatal flaw&quot; is a standard literary device. However, the device usually makes people feel sympathy when that

## Creating A Multi-Class Classifier Model

In [135]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)


In [136]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                mse = tf.metrics.mean_squared_error(label_ids, predicted_labels)

                return {
                    "Accuracy": accuracy,
                    "MSE": mse,
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


In [137]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [138]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE})

we will now create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [139]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
test_input_fn = bert.run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

## Training & Evaluating

In [ ]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [70]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\Users\yanni\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 60: eval_accuracy = 0.4125, false_negatives = 22.0, false_positives = 8.0, global_step = 60, loss = 1.3597462, true_negatives = 25.0, true_positives = 105.0


INFO:tensorflow:Saving dict for global step 60: eval_accuracy = 0.4125, false_negatives = 22.0, false_positives = 8.0, global_step = 60, loss = 1.3597462, true_negatives = 25.0, true_positives = 105.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60: bert\model.ckpt-60


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60: bert\model.ckpt-60


{'eval_accuracy': 0.4125,
 'false_negatives': 22.0,
 'false_positives': 8.0,
 'loss': 1.3597462,
 'true_negatives': 25.0,
 'true_positives': 105.0,
 'global_step': 60}

# Reference:
Most of the code has been taken from the following resource:

* https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

